<a href="https://colab.research.google.com/github/esther-pui/WOA7015-A2/blob/main/WOA7015_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

data = np.loadtxt('/Train_toydata.txt')

print(data)

[[ 0.77 -1.14  0.  ]
 [-0.33  1.44  0.  ]
 [ 0.91 -3.07  0.  ]
 [-0.37 -1.91  0.  ]
 [-1.84 -1.13  0.  ]
 [-1.5   0.34  0.  ]
 [-0.63 -1.53  0.  ]
 [-1.08 -1.23  0.  ]
 [ 0.39 -1.99  0.  ]
 [-1.26 -2.9   0.  ]
 [-5.27 -0.78  0.  ]
 [-0.49 -2.74  0.  ]
 [ 1.48 -3.74  0.  ]
 [-1.64 -1.96  0.  ]
 [ 0.45  0.36  0.  ]
 [-1.48 -1.17  0.  ]
 [-2.94 -4.47  0.  ]
 [-2.19 -1.48  0.  ]
 [ 0.02 -0.02  0.  ]
 [-2.24 -2.12  0.  ]
 [-3.17 -3.69  0.  ]
 [-4.09  1.03  0.  ]
 [-2.41 -2.31  0.  ]
 [-3.45 -0.61  0.  ]
 [-3.96 -2.    0.  ]
 [-2.95 -1.16  0.  ]
 [-2.42 -3.35  0.  ]
 [-1.74 -1.1   0.  ]
 [-1.61 -1.28  0.  ]
 [-2.59 -2.21  0.  ]
 [-2.64 -2.2   0.  ]
 [-2.84 -4.12  0.  ]
 [-1.45 -2.26  0.  ]
 [-3.98 -1.05  0.  ]
 [-2.97 -1.63  0.  ]
 [-0.68 -1.52  0.  ]
 [-0.1  -3.43  0.  ]
 [-1.14 -2.66  0.  ]
 [-2.92 -2.51  0.  ]
 [-2.14 -1.62  0.  ]
 [ 2.38  0.9   1.  ]
 [ 2.3   3.33  1.  ]
 [ 1.31  1.85  1.  ]
 [ 1.56  3.85  1.  ]
 [ 2.67  2.41  1.  ]
 [ 1.23  2.54  1.  ]
 [ 1.33  2.03  1.  ]
 [ 1.36  2.68